# Multi-Provider_Integration
LiteLLM 실습: 기본 사용법부터 API Gateway 구축까지  
이번 실습에서 LiteLLM을 사용하여 다양한 LLM을 일관된 방식으로 호출하고, 여러 모델을 하나로 묶는 API Gateway(Proxy)를 구축하는 방법을 실습합니다.
  
### 학습목표
1. LiteLLM을 사용해 OpenAI의 GPT 모델을 호출하는 방법을 익힙니다.

2. 동일한 코드로 모델만 변경하여 Google의 Gemini 모델을 호출해 봅니다.

3. LiteLLM Proxy를 사용하여 OpenAI, Google, 그리고 로컬 Ollama 모델을 통합하는 API Gateway를 구축하고 테스트합니다.


## LiteLLM 기본 사용법 : LLM 호출하기

In [1]:
from litellm import completion, acompletion, litellm

### API_KEY 설정
사용할 LLM의 API 키를 먼저 준비합니다.  
OPENAI_API_KEY와 GEMINI_API_KEY를 입력하겠습니다.

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

OpenAI API Key: ········
Google API Key: ········


LiteLLM의 `completion` 함수를 사용하여 OpenAI의 `gpt-4.0-mini` 모델을 호출합니다.  
이 코드는 "Hello, how are you?" 라는 질문을 `gpt-4.0-mini` 모델에 보내고, 모델의 응답을 출력합니다.  
LiteLLM은 내부적으로 이 요청을 OpenAI API에 맞게 변환하여 전송하고, 응답을 표준화된 형식으로 변환하여 출력합니다.

In [3]:
messages = [{"content": "Hello, how are you?", "role": "user"}]
response = litellm.completion(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Hello! I'm just a computer program, but I'm here and ready to help you. How are you today?


### 다른 LLM 호출 (Google) - 직접 코드를 완성해보세요!
동일한 `completion` 함수를 사용해 Google Gemini 모델을 호출해 봅시다.  
LiteLLM은 `model` 매개변수에 지정된 값을 기반으로 어떤 공급자의 API를 호출할지 결정합니다.  
여러 공급자의 모델을 일관된 코드 구조로 사용할 수 있습니다.  
`gemini/`라는 provider 프리픽스를 붙이면 LiteLLM이 Google AI Studio(Gemini) API 엔드포인트로 자동 라우팅합니다.  
이를 생략하면 LiteLLM이 이 모델명을 'OpenAI' 계열로 해석하거나 'provider를 찾을 수 없다'는 오류가 발생하므로 주의합니다.    

In [4]:
# Google 모델 호출 (gemini-1.5-flash)
# your code
google_api_key = os.environ.get("GOOGLE_API_KEY")

response_gemini = litellm.completion(
    model="gemini/gemini-1.5-flash-latest",
    messages=messages,
    api_key=google_api_key
)

# Gemini 모델의 응답 출력
print(response_gemini.choices[0].message.content)

print("고생하셨습니다.")

I am doing well, thank you for asking!  How are you today?

고생하셨습니다.


<details>
<summary>코드 예시 보기</summary>

```python
# 환경 변수에서 API 키 가져오기
google_api_key = os.environ.get("GOOGLE_API_KEY")

response_gemini = litellm.completion(
    model="gemini/gemini-1.5-flash-latest",
    messages=messages,
    api_key=google_api_key
)

# Gemini 모델의 응답 출력
print(response_gemini.choices[0].message.content)

print("고생하셨습니다.")

```
</details>

### 비동기 호출 실습
LLM 호출은 네트워크 지연이 발생할 수 있으므로, 비동기 방식으로 처리하는 것이 효율적일 수 있습니다.  
LiteLLM은 `acompletion` 함수를 통해 비동기 호출을 지원합니다.  
`asuncio.run()`을 사용해 비동기 함수를 실행합니다.  
여러 LLM 호출을 동시에 수행하거나, 웹 애플리케이션에서 비동기 처리가 필요한 경우 유용합니다.  

In [5]:
import asyncio
from litellm import acompletion

async def test_async_completion():
    user_message = "Hello, how are you?"
    messages = [{"role": "user", "content": user_message}]

    # 두 모델에 동시 요청
    gpt_task = acompletion(model="gpt-4o-mini", messages=messages, api_key=os.environ.get("OPENAI_API_KEY"))
    gemini_task = acompletion(model="gemini/gemini-1.5-flash-latest", messages=messages, api_key=os.environ.get("GOOGLE_API_KEY"))

    gpt_resp, gemini_resp = await asyncio.gather(gpt_task, gemini_task)

    return gpt_resp, gemini_resp

# gpt_resp, gemini_resp = asyncio.run(test_async_completion())
# asyncio.run()은 이미 실행 중인 이벤트 루프가 없을 때만 호출할 수 있으므로 주피터 노트북에서는 await test_async_completion() 사용
gpt_resp, gemini_resp = await test_async_completion()
print("GPT-4o-mini:", gpt_resp.choices[0].message.content)
print("Gemini:", gemini_resp.choices[0].message.content)

GPT-4o-mini: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
Gemini: I am doing well, thank you for asking!  How are you today?



In [ ]:
import time


async def compare_models(messages):
    models_to_test = ["gpt-4o-mini", "gemini/gemini-1.5-flash-latest"]

    # 1. 동기(순차) 실행 시간 측정
    start_time_sync = time.time()
    sync_responses = []
    for model in models_to_test:
        response = litellm.completion(model=model, messages=messages)
        sync_responses.append(response)
    end_time_sync = time.time()
    print(
        f"Sync execution time: {end_time_sync - start_time_sync:.2f} seconds")

    # 2. 비동기(병렬) 실행 시간 측정
    start_time_async = time.time()
    tasks = [acompletion(model=model, messages=messages)
             for model in models_to_test]
    async_responses = await asyncio.gather(*tasks)
    end_time_async = time.time()
    print(
        f"Async execution time: {end_time_async - start_time_async:.2f} seconds")

    # 3. 결과 출력
    for i, response in enumerate(async_responses):
        print(f"--- Response from {models_to_test[i]} ---")
        print(response.choices[0].message.content)
        print("-" * 20)

# 실행
await compare_models([{"role": "user", "content": "Tell me a short story about a robot who learns to paint."}])

### 스트리밍 응답 실습
대용량의 텍스트를 생성할 때는 스트리밍 응답을 사용하여 첫 번째 토큰부터 점진적으로 결과를 받을 수 있습니다.  
LiteLLM은 `stream=True` 매개변수를 통해 스트리밍 응답을 사용할 수 있습니다.  
`stream=True`를 설정하면 `completion` 함수는 제너레이터를 반환하며, for 루프를 통해 응답의 각 부분(chunk)을 순차적으로 처리할 수 있습니다.  
`chunk.choices[0].delta.content`를 통해 각 델타 내용을 얻을 수 있습니다.  

In [ ]:
# OpenAI 모델 스트리밍 호출
stream_response = completion(model="gpt-4o-mini", messages=messages, stream=True)
for chunk in stream_response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)
print()

## API Gateway/Proxy Server 구축하기

OpenAI(gpt-3.5-turbo), Google(gemini/gemini-1.5-flash-latest), 그리고 **로컬 LLM(llama3:8b)**을 하나로 묶는 Proxy를 구축해 보겠습니다.

### 로컬 LLM 환경 구축 (Ollama)

In [ ]:
# Ollama 설치 스크립트 실행 (Colab 환경용)
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Ollama 서버를 백그라운드에서 실행
# nohup: 터미널 세션이 끊겨도 프로세스가 계속 실행되도록 함
# &: 백그라운드에서 실행
import asyncio
import subprocess

command = "ollama serve"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Ollama 서버를 백그라운드에서 시작합니다... (약 5~10초 소요)")
await asyncio.sleep(10) # 서버가 준비될 때까지 잠시 대기
print("Ollama 서버가 준비되었습니다.")

In [ ]:
# 사용할 3B 모델(llama3.2:3b)을 다운로드합니다. (몇 분 정도 소요될 수 있습니다)
!ollama pull llama3.2:3b

In [ ]:
# 현재 Ollama에 로드된 모델 목록 확인
!ollama list

### Proxy 설정 파일 작성

LiteLLM Proxy는 config.yaml 파일을 통해 어떤 모델을 어떻게 라우팅할지 정의합니다.

- model_list: 사용할 모델의 목록을 정의합니다.

- litellm_settings: 여기서는 디버깅을 위해 로그를 출력하도록 설정합니다.

- 각 모델의 model_name은 우리가 API를 호출할 때 사용할 이름입니다.

- api_key, api_base 등은 각 모델에 맞게 설정합니다.

- Ollama 모델의 경우 api_base를 로컬에서 실행 중인 Ollama 서버 주소(http://localhost:11434)로 지정하는 것이 핵심입니다.

In [ ]:
%%writefile config.yaml
# %%writefile: 이 셀의 내용을 config.yaml 파일로 저장하는 Colab 명령어

model_list:
  - model_name: gpt-4o-mini  # API 호출 시 사용할 모델 이름
    litellm_params:
      model: openai/gpt-4o-mini # LiteLLM이 인식하는 실제 모델 이름
      api_key: os.environ/OPENAI_API_KEY # 환경 변수에서 키를 가져옴

  - model_name: gemini/gemini-flash
    litellm_params:
      model: gemini/gemini-1.5-flash-latest
      api_key: os.environ/GOOGLE_API_KEY

  - model_name: llama3.2-3b # 로컬 모델
    litellm_params:
      model: ollama/llama3.2:3b # ollama/ 접두사를 붙여 로컬 모델임을 명시
      api_base: http://localhost:11434 # 로컬 Ollama 서버 주소

litellm_settings:
  set_verbose: True # 디버깅을 위해 상세 로그 출력

### LiteLLM Proxy 서버 실행
이제 위에서 작성한 config.yaml 파일을 사용하여 Proxy 서버를 실행합니다.   
서버는 백그라운드에서 실행되며, nohup.out 파일에 로그가 기록됩니다.

In [ ]:
!pkill -f litellm

In [ ]:
# config.yaml 파일을 사용하여 LiteLLM Proxy를 백그라운드에서 실행
# 기본적으로 4000번 포트에서 실행됩니다.

get_ipython().system_raw('nohup litellm --config config.yaml > nohup.out 2>&1 &')


In [ ]:
# 서버가 시작될 시간을 잠시 기다립니다.
import time
time.sleep(10)

In [ ]:
# 4000번 포트를 사용하는 프로세스가 있는지 확인합니다.
!netstat -lntp | grep 4000

# 만약 처음 실행했을 때 아래와 같이 결과가 출력된다면 잠시 5초 정도 기다렸다가 다시 실행시켜보세요.
'''
(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)
'''

In [ ]:
# nohup.out 파일을 통해 서버 로그를 확인하여 정상적으로 실행되었는지 확인합니다.
!cat nohup.out

### Proxy에 요청 보내기
Proxy 서버가 성공적으로 실행되었습니다! 이제 **하나의 엔드포인트(http://localhost:4000)** 를 통해 3개의 다른 모델을 모두 호출해 보겠습니다.

테스트는 OpenAI의 openai 라이브러리를 그대로 사용합니다. base_url만 우리가 만든 Proxy 서버 주소로 변경해주면 됩니다.

In [ ]:
from openai import OpenAI

# 클라이언트 설정
# base_url을 우리 Proxy 서버 주소로 지정합니다.
# api_key는 아무 값이나 넣어도 상관없습니다 (인증은 Proxy가 담당).
client = OpenAI(
    api_key="anything",
    base_url="http://localhost:4000"
)

In [ ]:
import requests

response = requests.get("http://localhost:4000/v1/models")
print(response.json())

In [ ]:
# 1. GPT-4o-mini 호출 테스트
print("--- 1. Testing gpt-4o-mini ---")
response = client.chat.completions.create(
    model="gpt-4o-mini", # config.yaml에 정의한 model_name
    messages=[{"role": "user", "content": "What is your name?"}]
)
print(response.choices[0].message.content)
print("-" * 30)

In [ ]:
# 2. Gemini-1.5-Flash 호출 테스트
print("\n--- 2. Testing gemini/gemini-1.5-flash-latest ---")
response = client.chat.completions.create(
    model="gemini/gemini-1.5-flash-latest",
    messages=[{"role": "user", "content": "What is your name?"}]
)
print(response.choices[0].message.content)
print("-" * 30)

In [ ]:
# 3. 로컬 Llama3 호출 테스트
print("\n--- 3. Testing llama3.2:3b ---")
response = client.chat.completions.create(
    model="llama3.2-3b",
    messages=[{"role": "user", "content": "What is your name?"}]
)
print(response.choices[0].message.content)
print("-" * 30)

In [ ]:
# 4. 원하는 모델을 설정해서 다양한 대화를 나눠보세요.
# your code

In [ ]:
# 프로세스 확인 및 종료
!ps -ef | grep litellm

In [ ]:
# 프로세스 모두 종료
!pkill -f litellm